In [5]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from IPython.display import clear_output

#### Upload the dataset

In [6]:
train_and_valid_set = torchvision.datasets.FashionMNIST("./data", download=True, train=True, transform=
                                                transforms.ToTensor())
train_set, valid_set = torch.utils.data.dataset.random_split(train_and_valid_set, [48000, 12000])
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.ToTensor()) 

In [7]:
def update_data(BATCH_SIZE=64):
    train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(dataset=valid_set, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(dataset=test_set,batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, valid_loader, test_loader

#### Define the Final Network

In [11]:
class FinalNetwork(nn.Module):
    def __init__(self):

        super(FinalNetwork, self).__init__()
        self.flatten = nn.Flatten()
        input_features = 28*28
        num_nodes = 256
        num_classes = 10

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_features, num_nodes),   # layer 1
            nn.ReLU(),  # activation function 
            nn.Linear(num_nodes, num_nodes),        # layer 2
            nn.ReLU(),  # activation function 
            nn.Linear(num_nodes, num_classes)       # output layer
        )

    def forward(self, x):
        '''
        Execute the sequential stack for a flattened matrix and 
        return the raw logits 
        '''
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#### Training Loop and Validation Loop

In [12]:
def train_loop_modified(dataloader: torch.utils.data.dataloader.DataLoader, 
                model: FinalNetwork, 
                loss_function: torch.nn.modules.loss, 
                optimizer: torch.optim) -> float :

    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    correct = 0
    loss_acc = 0
    for batch, (image, image_label) in enumerate(dataloader):
        prediction = model(image)
        loss = loss_function(prediction, image_label)
        loss_acc += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        correct += (prediction.argmax(1) == image_label).type(torch.float).sum().item()
    correct /= size
    loss_acc /= num_batches
    return 100 * correct, loss_acc

def validation_loop_modified(dataloader: torch.utils.data.dataloader.DataLoader, 
                model: FinalNetwork, 
                loss_function: torch.nn.modules.loss):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for image, image_class in dataloader:
            pred = model(image)
            test_loss += loss_function(pred, image_class).item()
            correct += (pred.argmax(1) == image_class).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    return 100 * correct, test_loss

#### Run the Test

In [13]:
#----
# These parameters were chosen through model exploration
model = FinalNetwork()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)
epochs=17
batch_size=64
#----

train_loader, valid_loader, test_loader = update_data(batch_size)

for t in range(epochs):
    clear_output(wait=True)
    print(f"Currently on Epoch {t+1}....")
    train_accuracy, loss = train_loop_modified(train_loader, model, loss_function, optimizer)
    valid_accuracy,_ = validation_loop_modified(valid_loader, model, loss_function)

test_accuracy,_ = validation_loop_modified(test_loader, model, loss_function)

print(f"Training Accuracy is:\t {train_accuracy}")
print(f"Validation Accuracy is:\t {valid_accuracy}")
print(f"Test Accuracy is:\t {test_accuracy}")

Currently on Epoch 17....
Training Accuracy is:	 93.2625
Validation Accuracy is:	 90.15833333333333
Test Accuracy is:	 89.07000000000001
